In [ ]:
+!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U datasets
!pip install -U trl

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch
from datasets import load_dataset
from trl import SFTTrainer


In [ ]:
secret_hf = "YOUR_HUGGINGFACE_TOKEN"

In [ ]:
!huggingface-cli login --token $secret_hf

In [ ]:
base_model = "base_model_name"  # e.g., "meta-llama/Llama-2-7b-hf"
new_model = "new_model_name"  # e.g., "your_username/your_model_name"

In [ ]:
#Importing the dataset
from datasets import load_dataset
dataset = load_dataset("shailja/Verilog_GitHub", split="train[:1000]")  # Adjust the split as needed

In [ ]:
# Load base model
from transformers import BitsAndBytesConfig ,AutoModelForCausalLM, AutoTokenizer
import torch
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel, get_peft_model
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense',
        'fc1',
        'fc2',
    ]
)
model = get_peft_model(model, peft_config)
model.get_memory_footprint()

In [ ]:
#Hyperparamter
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="./batch-48-47001-48000",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="epoch",
    logging_steps=100,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    disable_tqdm=False,
    report_to="none",
)


In [ ]:
# Setting sft parameters
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

In [ ]:
output_dir = "D:/fine-tuned-model"  # Directory to save your model
trainer.model.save_pretrained(output_dir)
trainer.tokenizer.save_pretrained(output_dir)

In [ ]:
from huggingface_hub import notebook_login

# Authenticate with Hugging Face (if not already done)
notebook_login()

# Save the model locally first
trainer.model.save_pretrained(new_model)

# Push the model to the Hub, providing your Hugging Face token if necessary
trainer.push_to_hub(new_model, token='YOUR_HUGGINGFACE_TOKEN') # Replace 'YOUR_HUGGINGFACE_TOKEN' with your actual token

In [ ]:
prompt = "<|system|><|user|><|assistant|>"

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
result = pipe(prompt)
print(result[0]['generated_text'])

In [ ]:
!pip install transformers

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load your fine-tuned model and tokenizer
model_path = 'meghnareddy90/batch-7-6001-7000'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path,trust_remote_code=True)

# Prepare your prompt
prompt = "module my_module(input clk, input reset, output reg [7:0] out);\n"

# Tokenize and generate response
inputs = tokenizer(prompt, return_tensors='pt')
output_sequences = model.generate(
    input_ids=inputs['input_ids'],
    max_length=150,
    temperature=0.7,
    num_return_sequences=1,
)

# Decode and print the generated code
generated_code = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_code)
